<a href="https://colab.research.google.com/github/selcuk-yalcin/Convolutional-Neural-Network/blob/main/ConvolutionalNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F


In [13]:
# 1. Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [18]:
# 2. Define CNN
class BirdCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)

        self.conv1 = nn.Conv2d(3,   16, 3, padding=1)  # 32x32 -> pool: 16x16
        self.conv2 = nn.Conv2d(16,  32, 3, padding=1)  # 16x16 -> pool: 8x8
        self.conv3 = nn.Conv2d(32,  64, 3, padding=1)  # 8x8   -> pool: 4x4
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)  # 4x4   -> pool: 2x2

        self.fc1 = nn.Sequential(
            nn.Linear(128 * 2 * 2, 128),  # 512 -> 128
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32->16
        x = self.pool(F.relu(self.conv2(x)))  # 16->8
        x = self.pool(F.relu(self.conv3(x)))  # 8->4
        x = self.pool(F.relu(self.conv4(x)))  # 4->2
        x = x.view(x.size(0), -1)             # 128*2*2
        x = self.fc1(x)
        x = self.fc2(x)
        return x


In [23]:
# 3. Initialize model, loss, optimizer
net = BirdCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 4. Training loop
for epoch in range(20):  # train for 5 epochs (for speed)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[Epoch {epoch+1}, Batch {i+1}] loss: {running_loss/200:.3f}')
            running_loss = 0.0

print("Finished Training")

[Epoch 1, Batch 200] loss: 2.029
[Epoch 1, Batch 400] loss: 1.706
[Epoch 1, Batch 600] loss: 1.544
[Epoch 2, Batch 200] loss: 1.389
[Epoch 2, Batch 400] loss: 1.318
[Epoch 2, Batch 600] loss: 1.271
[Epoch 3, Batch 200] loss: 1.167
[Epoch 3, Batch 400] loss: 1.125
[Epoch 3, Batch 600] loss: 1.108
[Epoch 4, Batch 200] loss: 1.006
[Epoch 4, Batch 400] loss: 0.983
[Epoch 4, Batch 600] loss: 0.988
[Epoch 5, Batch 200] loss: 0.899
[Epoch 5, Batch 400] loss: 0.909
[Epoch 5, Batch 600] loss: 0.882
[Epoch 6, Batch 200] loss: 0.807
[Epoch 6, Batch 400] loss: 0.830
[Epoch 6, Batch 600] loss: 0.830
[Epoch 7, Batch 200] loss: 0.756
[Epoch 7, Batch 400] loss: 0.743
[Epoch 7, Batch 600] loss: 0.739
[Epoch 8, Batch 200] loss: 0.680
[Epoch 8, Batch 400] loss: 0.686
[Epoch 8, Batch 600] loss: 0.702
[Epoch 9, Batch 200] loss: 0.633
[Epoch 9, Batch 400] loss: 0.653
[Epoch 9, Batch 600] loss: 0.649
[Epoch 10, Batch 200] loss: 0.578
[Epoch 10, Batch 400] loss: 0.605
[Epoch 10, Batch 600] loss: 0.610
[Epoch 

In [27]:
# 5. Test accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on CIFAR-10 test images: {100 * correct / total:.2f}%')

Accuracy on CIFAR-10 test images: 72.23%
